In [23]:
# 使用InceptionV3,並進行微調
from keras.applications.inception_v3 import InceptionV3

In [24]:
img_size=150

In [25]:
con_base=InceptionV3(weights='imagenet',include_top=False,input_shape=(img_size,img_size,3))

In [26]:
from keras import layers
from keras import models
model=models.Sequential()
model.add(con_base)
model.add(layers.Flatten())
#model.add(layers.Dropout(0.3))
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(120,activation='softmax'))

In [27]:
count=0
for layer in con_base.layers:
    count+=1
print(count)

311


In [28]:
for layer in con_base.layers[:249]:
    layer.trainable = False
for layer in con_base.layers[249:]:
    layer.trainable = True

In [29]:
con_base.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 74, 74, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 74, 74, 32)   96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 74, 74, 32)   0           batch_normalization_95[0][0]     
_______________________________________________________________________________________

In [30]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 3, 3, 2048)        21802784  
_________________________________________________________________
flatten_3 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               4718848   
_________________________________________________________________
dense_6 (Dense)              (None, 120)               30840     
Total params: 26,552,472
Trainable params: 15,864,568
Non-trainable params: 10,687,904
_________________________________________________________________


In [31]:
from keras import optimizers
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

"""model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])"""

"model.compile(optimizer='rmsprop',\n              loss='categorical_crossentropy',\n              metrics=['accuracy'])"

In [32]:
from keras.preprocessing.image import ImageDataGenerator

train_path='data/images/train2'
val_path='data/images/val'

Train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode="nearest",
    horizontal_flip=True,
) 
Test_datagen = ImageDataGenerator(
    rescale=1./255
) 

In [33]:
Train_generator=Train_datagen.flow_from_directory(train_path,target_size=(img_size,img_size),batch_size=128,class_mode='categorical')

Found 25440 images belonging to 120 classes.


In [34]:
Val_generator=Test_datagen.flow_from_directory(val_path,target_size=(img_size,img_size),batch_size=128,class_mode='categorical')

Found 3600 images belonging to 120 classes.


In [ ]:
history=model.fit_generator(
    Train_generator,
    #steps_per_epoch=100,
    epochs=25,
    validation_data=Val_generator,
    validation_steps=50
)

Epoch 1/25
199/199 [==============================] - 167s 839ms/step - loss: 0.4300 - accuracy: 0.8733 - val_loss: 2.0417 - val_accuracy: 0.6851
Epoch 2/25
199/199 [==============================] - 159s 797ms/step - loss: 0.4307 - accuracy: 0.8732 - val_loss: 2.0801 - val_accuracy: 0.6870
Epoch 3/25
199/199 [==============================] - 159s 797ms/step - loss: 0.4279 - accuracy: 0.8734 - val_loss: 1.5781 - val_accuracy: 0.6831
Epoch 4/25
199/199 [==============================] - 170s 856ms/step - loss: 0.4279 - accuracy: 0.8734 - val_loss: 1.7616 - val_accuracy: 0.6875
Epoch 5/25
199/199 [==============================] - 170s 853ms/step - loss: 0.4030 - accuracy: 0.8820 - val_loss: 1.7031 - val_accuracy: 0.6875
Epoch 6/25
199/199 [==============================] - 168s 845ms/step - loss: 0.4060 - accuracy: 0.8774 - val_loss: 1.8843 - val_accuracy: 0.6844
Epoch 7/25
199/199 [==============================] - 169s 850ms/step - loss: 0.3997 - accuracy: 0.8817 - val_loss: 2.0154 -

In [ ]:
model.save('model_V7_50epochs2.h5')

In [ ]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
#plt.show()
plt.savefig("model_V7_50epochs_2-1.png")

In [ ]:
plt.clf()
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
#plt.show()
plt.savefig("model_V7_50epochs_2-2.png")